<a href="https://colab.research.google.com/github/sho6210/SecDev2023/blob/main/MLR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries

In [62]:
import sklearn
print(sklearn.__version__)
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from numpy.lib.function_base import vectorize
from sklearn.feature_extraction.text import CountVectorizer
from numpy.core.fromnumeric import size
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import tqdm
import time
import json
import re
import csv
import urllib.request
import shutil

1.2.2


# Data

In [63]:
# Load data
metric = 'cvssV3_attackVector'
df = pd.read_csv("https://raw.githubusercontent.com/sho6210/SecDev2023/main/data/cve_2018-2020_complete_dataset.csv", header=0)

# Make a data set

In [64]:
def DataSetMake(metric):
  X = pd.DataFrame(df[['Description']])
  y = pd.DataFrame(df[[metric]])
  train_sentence, test_sentence, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0, stratify=y)

  # Display data distribution.
  print("----------y_train:", y_train.value_counts())
  print("----------y_test:", y_test.value_counts())

  return train_sentence, test_sentence, y_train, y_test


# Natural Language Processing (train)

In [65]:
def NLP_train(train_sentence):
  # Natural Language Processing
  # Creating BoW features with sklearn's CountVectorizer
  vectorizer = CountVectorizer(stop_words="english")
  X_train = vectorizer.fit_transform(train_sentence['Description'].values)
  print('dimensions:', X_train.shape)
  
  return X_train, vectorizer

# Multinomial Logistic Regression (train)

In [66]:
def MLR_train(X_train, y_train): 
  # Create a classification model for MLR using vectorized features
  lr = LogisticRegression(C=0.1, random_state=0, n_jobs=-1)
  lr.fit(X_train, y_train)

  return lr



# Natural Language Processing (test)

In [67]:
def NLP_test(test_sentence, vectorizer):
  # Natural Language Processing
  # Creating BoW features with sklearn's CountVectorizer.
  X_test = vectorizer.transform(test_sentence['Description'].values)
  
  return X_test

# Multinomial Logistic Regression (test)

In [68]:
def MLR_test(metric, X_test, y_test, lr):
  # Test data to confirm accuracy.
  y_pred = lr.predict(X_test)
  cm = confusion_matrix(y_test, y_pred)

  # Branching when creating a table.
  if metric == 'cvssV3_attackVector':
    table = pd.DataFrame(cm, columns=['Predicted A', 'Predicted L', 'Predicted N', 'Predicted P'], index=['Actual A', 'Actual L', 'Actual N', 'Actual P'])
  elif metric == 'cvssV3_attackComplexity':
    table = pd.DataFrame(cm, columns=['Predicted H', 'Predicted L'], index=['Actual H', 'Actual L'])
  elif metric == 'cvssV3_privilegesRequired':
    table = pd.DataFrame(cm, columns=['Predicted H', 'Predicted L', 'Predicted N'], index=['Actual H', 'Actual L', 'Actual N'])
  elif metric == 'cvssV3_userInteraction':
    table = pd.DataFrame(cm, columns=['Predicted N', 'Predicted R'], index=['Actual N', 'Actual R'])
  elif metric == 'cvssV3_scope':
    table = pd.DataFrame(cm, columns=['Predicted C', 'Predicted U'], index=['Actual C', 'Actual U'])
  else:
    table = pd.DataFrame(cm, columns=['Predicted H', 'Predicted L', 'Predicted N'], index=['Actual H', 'Actual L', 'Actual N'])

  # Accuracy
  print("accuracy:", accuracy_score(y_test, y_pred))
  print(classification_report(y_test, y_pred, digits=4))
  print(table)
  print('-'*70)

  # Returns a list containing the prediction results and a data frame for testing.
  return y_pred



## Extraction of input tokens

In [70]:
def ExtractionTokens(vectorizer, lr):
  # Get the name of the feature (word).
  feature_names = vectorizer.get_feature_names_out() 
  
  # Binary time.
  if lr.coef_.shape[0] == 1:
    coef = lr.coef_[0] 

   # Extraction of positively and negatively impacted words.
    positive_words = [feature_names[i] for i in coef.argsort()[-10:]]
    negative_words = [feature_names[i] for i in coef.argsort()[:10]]

    print('Positive words:', positive_words)
    print('Negative words:', negative_words)


  # At three values or more.
  if lr.coef_.shape[0] != 1:

    # 分類区分ごとのポジティブとネガティブな単語の抽出
    coef_matrix = lr.coef_
    for i, class_name in enumerate(lr.classes_):
      coef = coef_matrix[i]
      positive_words = [feature_names[j] for j in coef.argsort()[-10:]]
      negative_words = [feature_names[j] for j in coef.argsort()[:10]]
      
      print("Class:", class_name)
      print("Positive words:", positive_words)
      print("Negative words:", negative_words)
    print('-' * 50)

# Example

In [98]:

# metrics = ['cvssV3_attackVector','cvssV3_attackComplexity','cvssV3_privilegesRequired','cvssV3_userInteraction',
#            'cvssV3_scope','cvssV3_confidentialityImpact','cvssV3_integrityImpact','cvssV3_availabilityImpact']

metric = 'cvssV3_attackVector'
train_sentence, test_sentence, y_train, y_test = DataSetMake(metric)

X_train, vectorizer = NLP_train(train_sentence)
lr = MLR_train(X_train, y_train)

X_test = NLP_test(test_sentence, vectorizer)
y_pred = MLR_test(metric, X_test, y_test, lr)

ExtractionTokens(vectorizer, lr)



----------y_train: cvssV3_attackVector
NETWORK                17045
LOCAL                   5046
ADJACENT_NETWORK         575
PHYSICAL                 297
dtype: int64
----------y_test: cvssV3_attackVector
NETWORK                17045
LOCAL                   5047
ADJACENT_NETWORK         575
PHYSICAL                 296
dtype: int64
dimensions: (22963, 36446)


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


accuracy: 0.9033227365762313
                  precision    recall  f1-score   support

ADJACENT_NETWORK     0.7989    0.4974    0.6131       575
           LOCAL     0.8613    0.7741    0.8154      5047
         NETWORK     0.9165    0.9643    0.9398     17045
        PHYSICAL     0.8382    0.3851    0.5278       296

        accuracy                         0.9033     22963
       macro avg     0.8537    0.6552    0.7240     22963
    weighted avg     0.9004    0.9033    0.8990     22963

          Predicted A  Predicted L  Predicted N  Predicted P
Actual A          286           30          256            3
Actual L           16         3907         1114           10
Actual N           54          546        16436            9
Actual P            2           53          127          114
----------------------------------------------------------------------
Class: ADJACENT_NETWORK
Positive words: ['authenticated', 'devices', 'segment', 'wireless', 'zte', 'network', 'hyper', 'bluetoot